<a href="https://colab.research.google.com/github/Fackor/Advanced_Machine_Learning_Course/blob/main/Data_Preparation_for_CoNLL2003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Upload the data to Colab.
2. Then run the following cells to prepare for the Regression task.

In [10]:
import pandas as pd

filename = "/content/train.txt"

In [11]:
df = pd.read_csv(filename, sep=" ", skiprows=0)
df

,-DOCSTART-,-X-,-X-.1,O
0,EU,NNP,B-NP,B-ORG
1,rejects,VBZ,B-VP,O
2,German,JJ,B-NP,B-MISC
3,call,NN,I-NP,O
4,to,TO,B-VP,O
...,...,...,...,...
204561,three,CD,I-NP,O
204562,Swansea,NN,B-NP,B-ORG
204563,1,CD,I-NP,O
204564,Lincoln,NNP,I-NP,B-ORG


In [12]:
# Rename columns.
'''
    The columns names are - 
    1. Token : These are the tokens that are present in the dataset.
    2. POS-Tag : These are the Part of Speech Tags
    3. SC-Tag : These are Syntactic Chunk Tags. (https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html)
    4. NE-Tag : These are the Named Entity Tags.
    
    More info on what the POS and SC represents -> (https://medium.com/greyatom/learning-pos-tagging-chunking-in-nlp-85f7f811a8cb)
'''
column_names = {'-DOCSTART-':'Token', 
                '-X-':'POS-Tag', 
                '-X-.1':'SC-Tag', 
                'O':'NE-Tag'}

df.rename(columns=column_names, errors="raise", inplace=True)

In [13]:
# Convert the IOB tags to O tags. (Remove B and I distinction.)
IOB_map =  {'B-ORG' : 'ORG',
            'I-ORG' : 'ORG',
            'B-PER' : 'PER',
            'I-PER' : 'PER',
            'B-LOC' : 'LOC',
            'I-LOC' : 'LOC',
            'B-MISC' : 'MISC',
            'I-MISC' : 'MISC',
            'O' : 'O'}
NE_tags = df['NE-Tag'].map(IOB_map)

df['NE-Tag'] = NE_tags
df

,Token,POS-Tag,SC-Tag,NE-Tag
0,EU,NNP,B-NP,ORG
1,rejects,VBZ,B-VP,O
2,German,JJ,B-NP,MISC
3,call,NN,I-NP,O
4,to,TO,B-VP,O
...,...,...,...,...
204561,three,CD,I-NP,O
204562,Swansea,NN,B-NP,ORG
204563,1,CD,I-NP,O
204564,Lincoln,NNP,I-NP,ORG


In [16]:
df.head(10)

,Token,POS-Tag,SC-Tag,NE-Tag
0,EU,NNP,B-NP,ORG
1,rejects,VBZ,B-VP,O
2,German,JJ,B-NP,MISC
3,call,NN,I-NP,O
4,to,TO,B-VP,O
5,boycott,VB,I-VP,O
6,British,JJ,B-NP,MISC
7,lamb,NN,I-NP,O
8,.,.,O,O
9,Peter,NNP,B-NP,PER
